# Model validation

### 7.1.1

In [2]:
import numpy as np

# requires data from exercise 1.5.1
from ex1_5_1 import *
from matplotlib.pyplot import figure, plot, show, xlabel, ylabel
from scipy.io import loadmat
from sklearn import model_selection
from sklearn.neighbors import KNeighborsClassifier

# This script crates predictions from three KNN classifiers using cross-validation

# Maximum number of neighbors
L = [1, 20, 80]

CV = model_selection.LeaveOneOut()
i = 0

# store predictions.
yhat = []
y_true = []
for train_index, test_index in CV.split(X, y):
    print("Crossvalidation fold: {0}/{1}".format(i + 1, N))

    # extract training and test set for current CV fold
    X_train = X[train_index, :]
    y_train = y[train_index]
    X_test = X[test_index, :]
    y_test = y[test_index]

    # Fit classifier and classify the test points (consider 1 to 40 neighbors)
    dy = []
    for l in L:
        knclassifier = KNeighborsClassifier(n_neighbors=l)
        knclassifier.fit(X_train, y_train)
        y_est = knclassifier.predict(X_test)

        dy.append(y_est)
        # errors[i,l-1] = np.sum(y_est[0]!=y_test[0])
    dy = np.stack(dy, axis=1)
    yhat.append(dy)
    y_true.append(y_test)
    i += 1

yhat = np.concatenate(yhat)
y_true = np.concatenate(y_true)
yhat[:, 0]  # predictions made by first classifier.
# Compute accuracy here.



Location of the iris.csv file: c:\Users\vietn\anaconda3\envs\ml\Lib\site-packages\dtuimldmtools\data\iris.csv
Crossvalidation fold: 1/150
Crossvalidation fold: 2/150
Crossvalidation fold: 3/150
Crossvalidation fold: 4/150
Crossvalidation fold: 5/150
Crossvalidation fold: 6/150
Crossvalidation fold: 7/150
Crossvalidation fold: 8/150
Crossvalidation fold: 9/150
Crossvalidation fold: 10/150
Crossvalidation fold: 11/150
Crossvalidation fold: 12/150
Crossvalidation fold: 13/150
Crossvalidation fold: 14/150
Crossvalidation fold: 15/150
Crossvalidation fold: 16/150
Crossvalidation fold: 17/150
Crossvalidation fold: 18/150
Crossvalidation fold: 19/150
Crossvalidation fold: 20/150
Crossvalidation fold: 21/150
Crossvalidation fold: 22/150
Crossvalidation fold: 23/150
Crossvalidation fold: 24/150
Crossvalidation fold: 25/150
Crossvalidation fold: 26/150
Crossvalidation fold: 27/150
Crossvalidation fold: 28/150
Crossvalidation fold: 29/150
Crossvalidation fold: 30/150
Crossvalidation fold: 31/150

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

### 7.1.2

In [3]:
from ex7_1_1 import *

from dtuimldmtools import jeffrey_interval

# Compute the Jeffreys interval
alpha = 0.05
[thetahatA, CIA] = jeffrey_interval(y_true, yhat[:, 0], alpha=alpha)

print("Theta point estimate", thetahatA, " CI: ", CIA)


Theta point estimate 0.956953642384106  CI:  (0.9194225123023887, 0.9831344032786383)


### 7.1.3

In [ ]:
from ex7_1_1 import *

from dtuimldmtools import mcnemar

# Compute the Jeffreys interval
alpha = 0.05
[thetahat, CI, p] = mcnemar(y_true, yhat[:, 0], yhat[:, 1], alpha=alpha)

print("theta = theta_A-theta_B point estimate", thetahat, " CI: ", CI, "p-value", p)


### 7.2.1

In [5]:
import numpy as np
import scipy.stats
import scipy.stats as st
import sklearn.tree

# requires data from exercise 1.5.1
from ex5_1_5 import *
from matplotlib.pyplot import figure, plot, show, xlabel, ylabel
from sklearn import model_selection
from sklearn.neighbors import KNeighborsClassifier

X, y = X[:, :10], X[:, 10:]
# This script crates predictions from three KNN classifiers using cross-validation

test_proportion = 0.2

X_train, X_test, y_train, y_test = model_selection.train_test_split(
    X, y, test_size=test_proportion
)

mA = sklearn.linear_model.LinearRegression().fit(X_train, y_train)
mB = sklearn.tree.DecisionTreeRegressor().fit(X_train, y_train)

yhatA = mA.predict(X_test)
yhatB = mB.predict(X_test)[:, np.newaxis]  #  justsklearnthings

# perform statistical comparison of the models
# compute z with squared error.
zA = np.abs(y_test - yhatA) ** 2

# compute confidence interval of model A
alpha = 0.05
CIA = st.t.interval(
    1 - alpha, df=len(zA) - 1, loc=np.mean(zA), scale=st.sem(zA)
)  # Confidence interval

# Compute confidence interval of z = zA-zB and p-value of Null hypothesis
zB = np.abs(y_test - yhatB) ** 2
z = zA - zB
CI = st.t.interval(
    1 - alpha, len(z) - 1, loc=np.mean(z), scale=st.sem(z)
)  # Confidence interval
p = 2 * st.t.cdf(-np.abs(np.mean(z)) / st.sem(z), df=len(z) - 1)  # p-value

p

array([0.00852969])

### 7.3.1

In [6]:
import scipy.stats as st
import sklearn.linear_model
import sklearn.tree

# requires data from exercise 1.5.1
from ex5_1_5 import *
from sklearn import model_selection

from dtuimldmtools import *
from dtuimldmtools.statistics.statistics import correlated_ttest

loss = 2
X,y = X[:,:10], X[:,10:]
# This script crates predictions from three KNN classifiers using cross-validation

K = 10 # We presently set J=K
m = 1
r = []
kf = model_selection.KFold(n_splits=K)

for dm in range(m):
    y_true = []
    yhat = []

    for train_index, test_index in kf.split(X):
        X_train, y_train = X[train_index,:], y[train_index]
        X_test, y_test = X[test_index, :], y[test_index]

        mA = sklearn.linear_model.LinearRegression().fit(X_train, y_train)
        mB = sklearn.tree.DecisionTreeRegressor().fit(X_train, y_train)

        yhatA = mA.predict(X_test)
        yhatB = mB.predict(X_test)[:, np.newaxis]  # justsklearnthings
        y_true.append(y_test)
        yhat.append( np.concatenate([yhatA, yhatB], axis=1) )

        r.append( np.mean( np.abs( yhatA-y_test ) ** loss - np.abs( yhatB-y_test) ** loss ) )

# Initialize parameters and run test appropriate for setup II
alpha = 0.05
rho = 1/K
p_setupII, CI_setupII = correlated_ttest(r, rho, alpha=alpha)

if m == 1:
    y_true = np.concatenate(y_true)[:,0]
    yhat = np.concatenate(yhat)

    # note our usual setup I ttest only makes sense if m=1.
    zA = np.abs(y_true - yhat[:,0] ) ** loss
    zB = np.abs(y_true - yhat[:,1] ) ** loss
    z = zA - zB

    CI_setupI = st.t.interval(1 - alpha, len(z) - 1, loc=np.mean(z), scale=st.sem(z))  # Confidence interval
    p_setupI = st.t.cdf(-np.abs(np.mean(z)) / st.sem(z), df=len(z) - 1)  # p-value

    print( [p_setupII, p_setupI] )
    print(CI_setupII, CI_setupI )

[0.0002860926897008202, 5.004483513549672e-65]
(-0.28765271788606883, -0.12465880710403474) (-0.22970625442913745, -0.18266688025409783)


### 7.4.3

In [7]:
# exercise 7.4.3
import importlib_resources
import numpy as np

# Load list of names from files

fmale = open(importlib_resources.files("dtuimldmtools").joinpath("data/male.txt"), "r")
ffemale = open(
    importlib_resources.files("dtuimldmtools").joinpath("data/female.txt"), "r"
)
mnames = fmale.readlines()
fnames = ffemale.readlines()
names = mnames + fnames
gender = [0] * len(mnames) + [1] * len(fnames)
fmale.close()
ffemale.close()

# Extract X, y and the rest of variables. Include only names of >4 characters.
X = np.zeros((len(names), 4))
y = np.zeros((len(names), 1))
n = 0
for i in range(0, len(names)):
    name = names[i].strip().lower()
    if len(name) > 3:
        X[n, :] = [
            ord(name[0]) - ord("a") + 1,
            ord(name[1]) - ord("a") + 1,
            ord(name[-2]) - ord("a") + 1,
            ord(name[-1]) - ord("a") + 1,
        ]
        y[n, 0] = gender[i]
        n += 1
X = X[0:n, :]
y = y[0:n, :]

N, M = X.shape
C = 2
attributeNames = ["1st letter", "2nd letter", "Next-to-last letter", "Last letter"]
classNames = ["Female", "Male"]

print("Ran Exercise 7.2.3")


Ran Exercise 7.2.3


### 7.4.4

In [9]:
# exercise 7.4.4w
import numpy as np
from ex7_4_3 import *
from sklearn import model_selection
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import OneHotEncoder

np.random.seed(2450)
y = y.squeeze()
0
# Naive Bayes classifier parameters
alpha = 1.0  # pseudo-count, additive parameter (Laplace correction if 1.0 or Lidtstone smoothing otherwise)
fit_prior = True  # uniform prior (change to True to estimate prior from data)

# K-fold crossvalidation
K = 10
CV = model_selection.KFold(n_splits=K, shuffle=True)

X = X[:, 0:4]  # using all 4 letters,
# for using e.g. only third letter or first and last try X[:,[2]] and X[:, [0,3]]

# We need to specify that the data is categorical.
# MultinomialNB does not have this functionality, but we can achieve similar
# results by doing a one-hot-encoding - the intermediate steps in in training
# the classifier are off, but the final result is corrent.
# If we didn't do the converstion MultinomialNB assumes that the numbers are
# e.g. discrete counts of tokens. Without the encoding, the value 26 wouldn't
# mean "the token 'z'", but it would mean 26 counts of some token,
# resulting in 1 and 2 meaning a difference in one count of a given token as
# opposed to the desired 'a' versus 'b'.
X = OneHotEncoder().fit_transform(X=X)

errors = np.zeros(K)
k = 0
for train_index, test_index in CV.split(X):
    # print('Crossvalidation fold: {0}/{1}'.format(k+1,K))

    # extract training and test set for current CV fold
    X_train = X[train_index, :]
    y_train = y[train_index]
    X_test = X[test_index, :]
    y_test = y[test_index]

    nb_classifier = MultinomialNB(alpha=alpha, fit_prior=fit_prior)
    nb_classifier.fit(X_train, y_train)
    y_est_prob = nb_classifier.predict_proba(X_test)
    y_est = np.argmax(y_est_prob, 1)

    errors[k] = np.sum(y_est != y_test, dtype=float) / y_test.shape[0]
    k += 1

# Plot the classification error rate
print("Error rate: {0}%".format(100 * np.mean(errors)))

print("Ran Exercise 7.2.4")


Error rate: 20.990614174303314%
Ran Exercise 7.2.4
